In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing import image


In [ ]:

# ✅ Clone GitHub Repository (Ensure dataset is inside)
#!git clone https://github.com/EASWAR17/dataset.git
DATASET_PATH = "/content/dataset/processedData"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

In [ ]:
# Define dataset paths
train_dir = os.path.join(DATASET_PATH, "train")
val_dir = os.path.join(DATASET_PATH, "val")
test_dir = os.path.join(DATASET_PATH, "test")

In [ ]:
# Image augmentation & preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Load dataset
train_generator = train_datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)


Found 699 images belonging to 2 classes.
Found 99 images belonging to 2 classes.
Found 202 images belonging to 2 classes.


In [ ]:
# Load pre-trained model (choose VGG16 or ResNet50)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification head
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(1, activation='sigmoid')(x)

In [ ]:
# Build final model
model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
epochs = 10
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 496s 23s/step - accuracy: 0.7093 - loss: 2.0548 - val_accuracy: 0.8788 - val_loss: 0.3186
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 482s 22s/step - accuracy: 0.9220 - loss: 0.1844 - val_accuracy: 0.9899 - val_loss: 0.0848
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 493s 23s/step - accuracy: 0.9461 - loss: 0.1184 - val_accuracy: 0.9899 - val_loss: 0.0861
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 514s 23s/step - accuracy: 0.9805 - loss: 0.0822 - val_accuracy: 0.9697 - val_loss: 0.1288
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 481s 22s/step - accuracy: 0.9544 - loss: 0.1112 - val_accuracy: 0.9899 - val_loss: 0.0721
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 479s 22s/step - accuracy: 0.9796 - loss: 0.0641 - val_accuracy: 0.9899 - val_loss: 0.0746
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 503s 23s/step - accuracy: 0.9751 - loss: 0.0638 - val_accuracy: 0.9899 - val_loss: 0.0489
Epoch 8/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 479s 22s/step - accuracy: 0.9571 - loss: 0.0856 - val_accuracy: 0.

In [ ]:
MODEL_SAVE_PATH = "/content/lung_cancer_model_vgg.h5"
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

Model saved to /content/lung_cancer_model_vgg.h5


In [ ]:

from tensorflow.keras.models import load_model

# Load the model
model = load_model("/content/lung_cancer_model_vgg.h5")

# Recompile the model with your desired settings
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [ ]:
# Plot training results
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
# Prediction function
def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    return "Cancerous" if prediction < 0.5 else "Non-Cancerous"

In [ ]:
/content/lung-cancer-detection/processedData/test/cancerous/adenocarcinoma150.png

In [ ]:
sample_image_path = os.path.join(DATASET_PATH, 'test/cancerous/large.cell.carcinoma70.png')  # Replace with your test image
print("Prediction:", predict_image(sample_image_path, model))
# generate_gradcam(model, sample_image_path)
# explain_with_shap(model, sample_image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 525ms/step
Prediction: Cancerous


In [ ]:
# # Grad-CAM Implementation
import cv2
def generate_gradcam(model, img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)


    grad_model = Model(inputs=model.input, outputs=[model.get_layer('block5_conv3').output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

    img = cv2.imread(img_path)
    img = cv2.resize(img, IMG_SIZE)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# def generate_gradcam(model, img_path):
#     img = image.load_img(img_path, target_size=(224, 224))  # Adjust size if needed
#     img_array = image.img_to_array(img) / 255.0
#     img_array = np.expand_dims(img_array, axis=0)

#     # Get model output from block5_conv3
#     grad_model = Model(inputs=model.input, outputs=[model.get_layer('block5_conv3').output, model.output])

#     with tf.GradientTape() as tape:
#         conv_outputs, predictions = grad_model(img_array)
#         loss = predictions[:, 0]  # Change index if multi-class

#         # Ensure gradient tracking
#         tape.watch(conv_outputs)

#     grads = tape.gradient(loss, conv_outputs)
#     pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

#     # Fix broadcasting issue
#     pooled_grads = tf.reshape(pooled_grads, (1, 1, -1))

#     conv_outputs = conv_outputs[0]
#     heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)

#     # Normalize heatmap safely
#     heatmap = np.maximum(heatmap, 0)
#     if np.max(heatmap) != 0:
#         heatmap /= np.max(heatmap)

#     # Load original image
#     img = cv2.imread(img_path)
#     img = cv2.resize(img, (224, 224))  # Ensure same size
#     heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))


#     # Convert heatmap to color
#     heatmap = np.uint8(255 * heatmap)
#     heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

#     # Overlay heatmap on image
#     superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)

#     # Display result
#     plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
#     plt.axis('off')
#     plt.show()

In [ ]:
# # SHAP Interpretation
# import shap
# def explain_with_shap(model, img_path):
#     img = image.load_img(img_path, target_size=IMG_SIZE)
#     img_array = image.img_to_array(img) / 255.0
#     img_array = np.expand_dims(img_array, axis=0)

#     # Convert a batch of images to a NumPy array as background data
#     background, _ = next(iter(train_generator))  # Extract batch
#     background = background[:50]  # Limit to 50 samples for efficiency

#     # Ensure background data is a NumPy array
#     background = np.array(background)

#     explainer = shap.GradientExplainer(model, background)
#     shap_values = explainer.shap_values(img_array)
#     shap.image_plot(shap_values, img_array)

import shap
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

# Ensure eager execution is disabled for SHAP compatibility
tf.compat.v1.disable_eager_execution()

def explain_with_shap(model, img_path, train_generator):
    # Load and preprocess image
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Generate background dataset (ensure correct shape)
    background, _ = next(iter(train_generator))
    background = np.array(background[:50])  # Take first 50 samples

    # Ensure background shape matches model input
    if background.shape[1:] != (224, 224, 3):
        background = np.array([cv2.resize(img, (224, 224)) for img in background])

    # Convert model to a Keras function (for compatibility)
    @tf.function
    def model_fn(x):
        return model(x)

    # Wrap model with SHAP GradientExplainer
    explainer = shap.GradientExplainer(model_fn, background)

    # Compute SHAP values
    shap_values = explainer.shap_values(img_array)

    # Display SHAP explanation
    shap.image_plot(shap_values, img_array)

# Example usage
# explain_with_shap(model, "path/to/image.png", train_generator)


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
/content/lung-cancer-detection/processedData/test/non-cancerous/normal108.png

In [ ]:
# Example prediction

sample_image_path = "/content/dataset/processedData/test/cancerous/large.cell.carcinoma88.png"
print("Prediction:", predict_image(sample_image_path, model))
generate_gradcam(model, sample_image_path)


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
explain_with_shap(model, sample_image_path, train_generator)


NameError: name 'train_generator' is not defined